In [1]:
import os
import glob
import math
import time
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import statsmodels.api as sm
import pylab as pl
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import umap
from sklearn import svm
from lightgbm import LGBMClassifier
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from k_means_constrained import KMeansConstrained

In [3]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [4]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')

## 1.3 Show the maximum return of train and test

In [5]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


### 1.3.1 Remove the Unnamed columns in dataframe

In [6]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

In [7]:
train_test = pd.concat([train, test], axis=0)

In [8]:
train_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET
0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.92759,2.974872e+05,2.974872e+05,15165.92759,4898.757333,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,2.322977e+04,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.19233,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,0.013314
1,6124.154099,1072.802927,41440.76212,41442.22458,70.456758,7.356050,70.379576,23229.76020,23230.10472,70.273618,7.389813,70.329906,23229.46908,23384.98219,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.441223,70.702624,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5100.350569,15788.07683,3.087904e+05,3.087904e+05,15788.07683,5100.350569,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,2.322977e+04,2.937218,-14.428210,75.204973,23317.46049,23309.42032,74.880368,689.670872,127.070357,36746.26762,31012.79786,-310.576721,4.532673,-47.045260,22053.23653,14626.73339,48.124991,-99.618253,-115.120518,7705.543821,22665.35143,-377.287072,-73.700375,-0.000448
2,5905.732593,1072.802927,41443.14358,41442.30403,70.422472,7.417794,70.376448,23229.48142,23229.61008,70.474265,7.388979,70.397301,23229.83396,23387.39168,83.418623,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.440201,70.686178,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5308.548086,16430.60795,3.204640e+05,3.204640e+05,16430.60795,5308.548086,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,2.322977e+04,141.231442,25.855392,75.184756,23248.26780,23307.91084,74.635569,634.523047,71.705965,28917.00549,24632.17456,419.071308,7.403187,118.846496,23430.24573,31251.55292,71.535567,53.482719,106.179152,37586.677270,23251.62576,261.098973,22.565621,0.000244
3,6029.325221,1072.802927,41440.76212,41442.25682,70.458533,7.366031,70.379221,23230.08433,23229.87971,70.288944,7.390120,70.370247,23229.81662,23390.02296,83.418623,11.615135,83.418623,23233.34325,23466.72590,83.418623,7.439191,70.692085,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5510.265781,17053.14128,3.317742e+05,7.571090e+04,17053.14128,5510.265781,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,2.322977e+04,-1.580478,-7.499746,75.184756,23248.26836,23307.91084,74.635569,583.001082,70.384022,25642.94219,23482.26269,-249.671869,7.388974,49.809681,23193.67720,15867.01579,70.369496,-12.169114,63.930236,10052.351290,23229.64352,-10.549985,4.656636,-0.000628
4,6142.360146,1072.802927,41443.14358,41442.46480,70.413623,7.411287,70.376788,23229.70975,23229.82255,70.467206,7.389910,70.386986,23229.87603,23392.66710,70.572881,11.615135,83.418623,23466.72590,23466.72590,83.418623,7.438244,70.680386,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,1152.667997,17699.01445,3.435085e+05,3.435085e+05,17699.01445,5719.546213,70.376262,2.322977e+04,23229.77107,70.376262,7.389056,70.376262,23229.77107,2.322977e+04,149.144800,26.789196,74.904988,23229.77108,23260.32245,70.506980,566.549008,120.694073,25765.82131,24618.79392,363.188022,7.389057,78.826922,23235.86490,31790.06425,120.694068,42.971377,145.572170,37109.895810,24143.94971,188.639704,31.863254,0.003811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [9]:
train_test_np = train_test.iloc[:, :66].to_numpy()

In [11]:
kmeans = KMeans(n_clusters=5000, random_state=0).fit(train_test_np)
train_np = train_test.iloc[:90000, :66].to_numpy()
test_np = train_test.iloc[90000:, :66].to_numpy()

train_pred = kmeans.predict(train_np)
test_pred = kmeans.predict(test_np)
train['pred_class'] = train_pred
test['pred_class'] = test_pred

In [12]:
quantiles_num = []
train_sum_returns = []
test_sum_returns = []

models = []

for i in range(5000):
    
    train_seg = train[train['pred_class']==i]
    test_seg = test[test['pred_class']==i]
    if train_seg.shape[0] == 1:
        continue
    if test_seg.shape[0] < 1:
        continue
    reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
    models.append(reg)
    
    pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

    pred_test = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

    train_res = np.sum(train_seg['TARGET'][pred>0])
    test_res = np.sum(test_seg['TARGET'][pred_test>0])
    
    train_sum_returns.append(train_res)
    test_sum_returns.append(test_res)

ValueError: Found array with 0 sample(s) (shape=(0, 66)) while a minimum of 1 is required.

In [ ]:
print(sum(train_sum_returns))
print(sum(test_sum_returns))